In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()) + "/scripts")
sys.path.append(os.path.dirname(os.getcwd()) + "/scripts/pseudo_labels")
sys.path.append(os.path.dirname(os.getcwd()) + "/scripts/pseudo_labels/proto")
print(os.path.dirname(os.getcwd()))

from label_generation import VoxelMap
from label_generation import RayCaster
from label_generation import Visualizer3D

import numpy as np

/home/jonfrey/catkin_ws/src/Kimera-Interfacer/kimera_interfacer


In [2]:
import struct

from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointCloud2, PointField
from std_msgs.msg import Header

In [3]:

from collections import OrderedDict

ORDERED_DICT = OrderedDict(
  [
    ("unlabeled", (0, 0, 0)),
    ("wall", (174, 199, 232)),
    ("floor", (152, 223, 138)),
    ("cabinet", (31, 119, 180)),
    ("bed", (255, 187, 120)),
    ("chair", (188, 189, 34)),
    ("sofa", (140, 86, 75)),
    ("table", (255, 152, 150)),
    ("door", (214, 39, 40)),
    ("window", (197, 176, 213)),
    ("bookshelf", (148, 103, 189)),
    ("picture", (196, 156, 148)),
    ("counter", (23, 190, 207)),
    ("blinds", (178, 76, 76)),
    ("desk", (247, 182, 210)),
    ("shelves", (66, 188, 102)),
    ("curtain", (219, 219, 141)),
    ("dresser", (140, 57, 197)),
    ("pillow", (202, 185, 52)),
    ("mirror", (51, 176, 203)),
    ("floormat", (200, 54, 131)),
    ("clothes", (92, 193, 61)),
    ("ceiling", (78, 71, 183)),
    ("books", (172, 114, 82)),
    ("refrigerator", (255, 127, 14)),
    ("television", (91, 163, 138)),
    ("paper", (153, 98, 156)),
    ("towel", (140, 153, 101)),
    ("showercurtain", (158, 218, 229)),
    ("box", (100, 125, 154)),
    ("whiteboard", (178, 127, 135)),
    ("person", (120, 185, 128)),
    ("nightstand", (146, 111, 194)),
    ("toilet", (44, 160, 44)),
    ("sink", (112, 128, 144)),
    ("lamp", (96, 207, 209)),
    ("bathtub", (227, 119, 194)),
    ("bag", (213, 92, 176)),
    ("otherstructure", (94, 106, 211)),
    ("otherfurniture", (82, 84, 163)),
    ("otherprop", (100, 85, 144)),
  ]
)

SCANNET_COLOR_MAP = {i: v for i, v in enumerate(ORDERED_DICT.values())}



In [4]:
import  google.protobuf as pb

In [5]:
from google.protobuf.internal.decoder import _DecodeVarint32
import numpy as np
from Layer_pb2 import LayerProto
from Block_pb2 import BlockProto

EPS = 1e-2

def get_a_b_c_from_linear(index, voxels_per_side):
  a = index % (voxels_per_side)
  b = ((index - a) / (voxels_per_side)) % (voxels_per_side)
  c = ((index - a) - (voxels_per_side * b)) / (voxels_per_side ** 2)
  return (int(a), int(b), int(c))


class VoxelData():
    def __init__(self, fn):
       
        with open(fn, "rb") as f:
            buf = f.read()
            n = 0

            msg_len, new_pos = _DecodeVarint32(buf, n)
            n += new_pos

            num_protos = buf[n] 
            start = n

            header = LayerProto()
            msg_len, n = _DecodeVarint32(buf, n)
            msg_buf = buf[n : n + msg_len]
            n += msg_len
            header.ParseFromString(msg_buf)

            num_blocks = num_protos

            blocks = []
            
            while True:
                try:

                    block = BlockProto()
                    start = n

                    msg_len, n = _DecodeVarint32(buf, n)
                    msg_buf = buf[n : n + msg_len]
                    n += msg_len
                    block.ParseFromString(msg_buf)

                    if len(blocks) == 0:
                        tmp = block.voxel_size
                        blocks.append(block) 
                    else:
                        if block.voxel_size == tmp:
                            blocks.append(block) 
                        else:
                            break
                except:
                    print("Failed")
                    print("Blocks done: ",len(blocks))
                    break
                
        self.blocks = blocks
        self.header = header
    
    def to_numpy(self):
        ls = []
        for b in self.blocks:
            ls.append( [ b.origin_x,b.origin_y,b.origin_z] )
        
        vps = self.header.voxels_per_side
        centers =  np.array( ls ) 
        block_length  = ( self.header.voxel_size * self.header.voxels_per_side ) 
        cells = np.round( ( centers.max(axis = 0) - centers.min(axis = 0) ) / block_length )
        
        block_origin = centers.min(axis = 0)
        
        data = np.zeros( np.uint32( (cells+1) * vps ) )
        weight = np.zeros( np.uint32( (cells+1) * vps ) )
        rgba = np.zeros( np.uint32( (cells+1) * vps ) )
        for b in range (len( self.blocks)):
            index= np.round( (centers[b]- block_origin ) / block_length )
            voxel_start = np.uint32( index * vps ) 
            voxel_stop = np.uint32( (index+1) * vps )
            
            for i in range(0, len(self.blocks[b].voxel_data) ,3 ):
                _a,_b,_c = get_a_b_c_from_linear( i/3, vps)
                data[ voxel_start[0]+ _a,voxel_start[1]+_b,_c+voxel_start[2] ] = self.blocks[b].voxel_data[i]
                weight[ voxel_start[0]+ _a,voxel_start[1]+_b,_c+voxel_start[2] ] = self.blocks[b].voxel_data[i+1]
                rgba[ voxel_start[0]+ _a,voxel_start[1]+_b,_c+voxel_start[2] ] = self.blocks[b].voxel_data[i+2]
            
        self.data = data
        self.weight = weight
        self.rgba = rgba
        
# scene0000_00_labels_iter2_5cm_rep02_reprojected_predict_tsdf_esdf_layers.vxblx
# 10 scene0000_00_labels_gt_10cm_reprojected_predict_tsdf_esdf_layers.vxblx
vd = VoxelData( "/home/jonfrey/Datasets/output_kimera_semantics/scene0000_00_labels_iter2_5cm_rep02_reprojected_predict_tsdf_esdf_layers.vxblx")
vd.to_numpy()

Failed
Blocks done:  100


In [6]:
from struct import pack, unpack
s = vd.data.shape
distance = np.zeros_like(vd.data).view(np.float32)

for i in range(s[0]):
    for j in range(s[1]):
        for k in range(s[2]):
            val = int( vd.data[i,j,k] )
            b = pack('I', val)
            distance[i,j,k] = unpack('f', b)[0]
            
s = vd.data.shape
weight = np.zeros_like(vd.weight).view(np.float32)
for i in range(s[0]):
    for j in range(s[1]):
        for k in range(s[2]):
            val = int( vd.weight[i,j,k] )
            b = pack('I', val)
            weight[i,j,k] = unpack('f', b)[0]

In [7]:
import rospy
rospy.init_node('turtlebot_talktest', anonymous=True)

from sensor_msgs.msg import PointCloud2
import std_msgs.msg
import sensor_msgs.point_cloud2 as pcl2

pub_cloud = rospy.Publisher("points", PointCloud2)
rospy.sleep(1.)
cloud_points = [[1.0, 1.0, 0.0],[1.0, 2.0, 0.0]]
header = std_msgs.msg.Header()
header.stamp = rospy.Time.now()
header.frame_id = 'map'
#create pcl from points
scaled_polygon_pcl = pcl2.create_cloud_xyz32(header, cloud_points)
#publish    
rospy.loginfo("happily publishing sample pointcloud.. !")
pub_cloud.publish(scaled_polygon_pcl)

INFO - 2021-09-08 21:13:08,816 - topics - topicmanager initialized
<ipython-input-7-4c5dbc15abf9>:8: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  pub_cloud = rospy.Publisher("points", PointCloud2)


[INFO] [1631128389.941203]: happily publishing sample pointcloud.. !


In [8]:
# 10cm  scene0000_00_labels_gt_10cm_reprojected_serialized.data
#scene0000_00_labels_iter2_5cm_rep02_reprojected_serialized.data
voxel_map = VoxelMap("/home/jonfrey/Datasets/output_kimera_semantics/scene0000_00_labels_iter2_5cm_rep02_reprojected_serialized.data",
                     (320,640), 1)

In [9]:
voxel_map._voxels.shape, weight.shape

from matplotlib import cm

mask = voxel_map._voxels.sum(axis=3) != 0
res =  voxel_map._voxels - voxel_map._voxels.min(axis=3)[:,:,:,None] 
res[mask] = res[mask] / res[mask].sum(axis=1)[:,None]
cert = res.max(axis=3)
col = res.argmax(axis=3)

from matplotlib import cm
colors = cm.hot(range(256))

In [ ]:
import struct

from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointCloud2, PointField
from std_msgs.msg import Header

points = []
lim = 8
size= 0.05
#mask = vd.data != 0 
#(vd.data[mask].mean() < vd.data[mask].mean() )
#mea = vd.data[mask].mean()
#mask = ( vd.data < mea ) * ( vd.data != 0 ) 
#mask = (vd.weight > 0.001) * (np.abs(vd.data < 0.3 ))
val = np.abs(distance).mean()
mask = ((weight > 0.004) )  * (np.abs(distance) < size*1.5 ) 
indis = np.where(mask)

for _i in range(0, indis[0].shape[0]):
    i = indis[0][_i]
    j = indis[1][_i]
    k = indis[2][_i]
    if not mask[i,j,k]:
        continue

    x = float(i) * size
    y = float(j) * size
    z = float(k) * size
    
    
    r,g,b,a = 255,0,0,255
    try:
        r,g,b = SCANNET_COLOR_MAP[col[i,j,k]]
    except:
        pass
    
    try:
        pass
        ind = int( cert[i,j,k] * 255)
        r,g,b,a = np.int32( np.array( cm.get_cmap("plasma")(ind) ) * 255 )
        #g = int(  ind )
        #r = int( (255-ind) )
        #b = 0
        #a = 255
    except:
        pass 
    
    rgb = struct.unpack('I', struct.pack('BBBB', b, g, r, a))[0]
    
    pt = [x, y, z, rgb]
    points.append(pt)
    
fields = [PointField('x', 0, PointField.FLOAT32, 1),
          PointField('y', 4, PointField.FLOAT32, 1),
          PointField('z', 8, PointField.FLOAT32, 1),
          # PointField('rgb', 12, PointField.UINT32, 1),
          PointField('rgba', 12, PointField.UINT32, 1),
          ]

header = Header()
header.frame_id = "world"
pc2 = point_cloud2.create_cloud(header, fields, points)
print(len(points))
while not rospy.is_shutdown():
    pc2.header.stamp = rospy.Time.now()
    pub_cloud.publish(pc2)
    print("working")
    rospy.sleep(1.0)

110679
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
w

working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import OrderedDict

cmaps = OrderedDict()

cmaps['Perceptually Uniform Sequential'] = [
             'plasma']

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient)).T
gradient = 1-gradient
fig, axs = plt.subplots(nrows=1, figsize=(0.3, 6.4))
axs.imshow(gradient, aspect='auto',cmap=plt.get_cmap('plasma'))
#
axs.set_yticks([0,51,102,153,204,255], minor=False)
axs.get_xaxis().set_visible(False)
plt.show()